회귀분석용 서울시 동별, 년도별 유동인구, 거주인구, 직장인구 크롤링

출처:https://golmok.seoul.go.kr/main.do

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name= fm.FontProperties(fname='C:\\Windows\\Fonts\\malgun.ttf').get_name()
plt.rc('font',family=font_name)

import matplotlib as mlp
mlp.rcParams['axes.unicode_minus']=False

from bs4 import BeautifulSoup
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# pip install webdriver-manager
# from webdriver_manager.chrome import ChromeDriverManager # 크롬 버전 알아서 맞춰줌
from selenium.webdriver.chrome.service import Service

import time

In [14]:
driver = webdriver.Chrome("chromedriver")
driver.get("https://golmok.seoul.go.kr/stateArea.do") # 큰 화면으로 실행

driver.find_element(By.ID, "population").click()

In [20]:
# 여기 년도만 바꿔서 실행
Year= 2020
year= driver.find_element(By.ID, "selectYear")
year.click()
tag= year.find_elements(By.TAG_NAME, 'option')
for i in tag:
#     print(i.text)
    if i.text==f'{Year}년':
        i.click()

qu= driver.find_element(By.ID, "selectQu")
qu.click()
tag= qu.find_elements(By.TAG_NAME, 'option')  
for i in tag:
#     print(i.text)
    if i.text=='4분기':
        i.click()

qucon= driver.find_element(By.ID, "selectQuCondition")
qucon.click()
tag= qucon.find_elements(By.TAG_NAME, 'option')  
for i in tag:    
#     print(i.text)
    if i.text=='전분기':
        i.click()

driver.find_element(By.ID, "presentSearch").click()

time.sleep(3)

구=[]
동=[]
유동인구2=[]
주거인구2= []
직장인구2=[]

tbody= driver.find_element(By.TAG_NAME, 'tbody')

branchs= tbody.find_elements(By.CLASS_NAME, 'branch')

trs= tbody.find_elements(By.TAG_NAME, 'tr')

for tr in trs:
    if tr.get_attribute('data-tt-parent-id') =='1':
        gu= tr.find_element(By.TAG_NAME, 'a')
        gu.click()
        Texts= tr.text.replace(',','').split()

    elif tr.get_attribute('data-tt-id') != '1':
        구.append(Texts[0])
        texts= tr.text.replace(',','').split()
        동.append(texts[0])
        s_p= [int(texts[1]), int(texts[4]), int(texts[7])]
        l_p= [int(texts[2]), int(texts[5]), int(texts[8])]
        w_p= [int(texts[3]), int(texts[6]), int(texts[9])]
        유동인구2.append(sum(s_p)/3)
        주거인구2.append(sum(l_p)/3)
        직장인구2.append(sum(w_p)/3)

tag= driver.find_elements(By.TAG_NAME, 'option')

time.sleep(3)
for i in tag:
#     print(i.text)
    if i.text==f'{Year}년':
        i.click()
    if i.text=='3분기':
        i.click()
    if i.text=='전분기':
        i.click()
driver.find_element(By.ID, "presentSearch").click()
time.sleep(2)

유동인구1=[]
주거인구1=[]
직장인구1=[]
trs= driver.find_elements(By.CLASS_NAME, 'branch')
tbody= driver.find_element(By.TAG_NAME, 'tbody')

# trs= driver.find_elements(By.CLASS_NAME, 'branch')
branchs= tbody.find_elements(By.CLASS_NAME, 'branch')
trs= tbody.find_elements(By.TAG_NAME, 'tr')

for tr in trs:
    if tr.get_attribute('data-tt-parent-id') =='1':
        gu= tr.find_element(By.TAG_NAME, 'a')
        gu.click()

    elif tr.get_attribute('data-tt-id') != '1':
        texts= tr.text.replace(',','').split()

        유동인구1.append(int(texts[1]))
        주거인구1.append(int(texts[2]))
        직장인구1.append(int(texts[3]))

data1={'구':구,'동':동, '1분기': 유동인구1,'2~4분기': 유동인구2}
data2= {'구':구,'동':동, '1분기': 주거인구1, '2~4분기': 주거인구2}
data3= {'구':구,'동':동, '1분기': 직장인구1, '2~4분기':직장인구2}
df1= pd.DataFrame(data1)
df2= pd.DataFrame(data2)
df3= pd.DataFrame(data3)

df1['평균유동인구']= (df1['1분기']+df1['2~4분기'])/2
df1.drop(['1분기','2~4분기'], axis=1, inplace=True)

df2['평균주거인구']= (df2['1분기']+df2['2~4분기'])/2
df2.drop(['1분기','2~4분기'], axis=1, inplace=True)

df3['평균직장인구']= (df3['1분기']+df3['2~4분기'])/2
df3.drop(['1분기','2~4분기'], axis=1, inplace=True)

df1.to_csv(f'{Year}유동인구.csv')
df2.to_csv(f'{Year}주거인구.csv')
df3.to_csv(f'{Year}직장인구.csv')

In [7]:
tag

[<selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="91eb0c51-ae23-4b51-b4d9-831e6c0cd513")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="68b18785-0349-4dbe-b18d-7ac711a3cfb0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="0d6bc9b5-7408-4594-affc-25ad9a425ab4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="a892476b-39fa-4807-a3cf-57148525046d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="f80fe8bf-9d09-472c-b74b-67d855124156")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="929b3106-c052-4b88-bb06-4e7f19b9d446")>,
 <selenium.webdriver.remote.webelement.WebElement (session="83588d68abd911ed107ccda15d683e64", element="cfcb2e0b-7656-4fcf-b4ea-51

In [23]:
len(유동인구1)*5

2120